In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Load the dataset
data = pd.read_csv('sensor.csv')

# Fill missing values using forward fill
data = data.fillna(method='ffill')

# Labels
y = data['machine_status']

data['timestamp'] = pd.to_datetime(data['timestamp'])

# Extract month, day, minute, and hour from the timestamp
data['month'] = data['timestamp'].dt.month
data['day'] = data['timestamp'].dt.day
data['minute'] = data['timestamp'].dt.minute
data['hour'] = data['timestamp'].dt.hour


data=data.drop(['timestamp','Unnamed: 0','sensor_15'],axis=1)

# Drop the 'machine_status' column to get the features
X = data.drop(['machine_status', ], axis=1)

# Check the number of unique classes
num_classes = y.nunique()

if len(X) < num_classes:
    raise ValueError(f"Not enough samples to create a test set with each class present. Total samples: {len(X)}, Classes: {num_classes}")

# Adjusted test size, ensuring it's greater than or equal to the number of classes
test_size = 0.25  # 25% might be fine here

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)

# Train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, zero_division=0))

# Function to classify a new input
def check_value(input_value):
    # Ensure input_value is formatted as a 2D array for prediction
    input_df = pd.DataFrame([input_value], columns=X.columns)
    prediction = model.predict(input_df)
    return prediction[0]

# Test data
tdf = pd.read_csv('sensor.csv')

# Assuming 'logsTest.csv' contains sensor data similar to 'sensor.csv' (all feature columns except 'machine_status')
new_values = tdf[X.columns].values

# Testing new inputs
for value in new_values:
    result = check_value(value)
    print(f'Input: {value}, Classification: {result}')


              precision    recall  f1-score   support

      BROKEN       0.00      0.00      0.00         2
      NORMAL       1.00      1.00      1.00     51459
  RECOVERING       1.00      1.00      1.00      3619

    accuracy                           1.00     55080
   macro avg       0.67      0.67      0.67     55080
weighted avg       1.00      1.00      1.00     55080



KeyError: "['month', 'day', 'minute', 'hour'] not in index"